In [1]:
##########################################################################################
# Abraham Tishelman-Charny                                                               #
# 3 March 2021                                                                           #
#                                                                                        #
# The purpose of this notebook is to validate Resonant HH UL gridpacks. This can         #
# also be used to study any MadGraph 5 gridpacks.                                        #
##########################################################################################

# Thanks to:
# https://pydoc.net/pylhe/0.0.2/pylhe/ ##-- pylhe 
# https://github.com/scikit-hep/pylhe/blob/master/examples/zpeak.ipynb

In [2]:
##-- Imports 
import pylhe 
import ROOT
import math
from matplotlib import pyplot as plt 
import numpy as np 
import os 
import pandas as pd 
import pickle 
from ROOT import TLorentzVector

Welcome to JupyROOT 6.12/06


In [3]:
##-- Get LHE Files assuming directory tree structure: LHE_Files/<gridpack_Type>/<files>
gridpack_Type = "GF_XYH"
#gridpack_Type = "GF_Spin-0"
# gridpack_Type = "GF_Spin-2"
# gridpack_Type = "VBF_Spin-0"
# gridpack_Type = "VBF_Spin-2"

files = ["LHE_Files/%s/%s"%(gridpack_Type,file) for file in os.listdir('./LHE_Files/%s'%(gridpack_Type)) if file.endswith('.lhe')] 
for file in files:
    print("Found File:",file)
print("N files:",len(files))

Found File: LHE_Files/GF_XYH/cmsgrid_final.lhe
N files: 1


In [10]:
##-- Create Pandas dataframe for each file 

##-- Parameters 
##################
debug = 0      
maxEvents = 100
lessPoints = 0
##################

##-- Functions for Event Loop
def GetPt(p):
    px = p.px
    py = p.py 
    pt = np.sqrt(px**2 + py**2)
    return pt

def GetDR(eta1, eta2, phi1, phi2):
    DR = np.sqrt((eta1 - eta2)**2 + (phi1 - phi2)**2)
    return DR

##-- For each LHE file (one per gridpack)
for i_file, file in enumerate(files):

    ##-- File Printout
    print("On file:",file)
    
    ##-- Prepare dictionary of lists for pandas dataframe 
    d = {}
    
    ##-- Define particles based on gridpack type 
    particleDictionary = {
        "GF_XYH" : ["X","Lead_H","Sublead_H","Lead_g","Sublead_g"], 
    }
    
    ##-- Define particles to determine lead / sublead for 
    doubleParticleDict = {
        "GF_XYH" : ["H", "g"], 
    }    
    
    particle_Names = particleDictionary[gridpack_Type]
    doubleParticles = doubleParticleDict[gridpack_Type]
    
    ##-- Define variables to save for each particle
    variables = ["px", "py", "pz", "e", "m", "spin", "status", "lifetime", "pt"]
    ##-- possible variables: ['id','status','color1','color2','px','py','pz','e','m','lifetime','spin']  
    
    ##-- Initialize dictionary lists 
    for p in particle_Names:
        for v in variables:
            columnName = "%s_%s"%(p,v)
            d[columnName] = [] 
    
    ##-- Save eta and phi for X, both Higgs 
    for ang_particle in ["X", "Lead_H", "Sublead_H","Lead_g","Sublead_g"]:
        for ang_var in ["eta", "phi"]:
            exec("d['%s_%s'] = []"%(ang_particle, ang_var))
            
    ##-- For both GF and VBF: Initialize lists to save event by event pdgIds, DR(H, H), Deta(HH), Dphi(HH)
    d["pdgIds"] = []
    d["DR_HH"] = []
    d["HH_Deta"] = []
    d["HH_Dphi"] = []
    
    ##-- For Each Event
    print("Looping Events ...")
    for ie, event in enumerate(pylhe.readLHE(file)):
        if(ie%1000 == 0): print("On Event %s"%(ie))
        if(ie == maxEvents): 
            print("STOPPING at %s events"%(maxEvents))
            break 
        particles = event.particles
        pdgIds = []
        foundFirstHiggs, foundFirstGluon = 0, 0  
        foundFirstIncomingQuark, foundFirstOutgoingQuark = 0, 0
        
        ##-- Get Particles
        for particle in particles:
            pdgId = particle.id
            pdgIds.append(pdgId)
            status = particle.status
            
            ##-- Lots of particle information if debugging
            if(debug): 
                print("")
                print("pdgId:",pdgId)
                print("mass:",particle.m)
                print("mother1:",particle.mother1)
                print("mother2:",particle.mother2)
                print("color1:",particle.color1)
                print("color2:",particle.color2)                
                print("status:",particle.status)
                print("lifetime:",particle.lifetime)  
                print("px:",particle.px)  
                print("py:",particle.py)  
                print("pz:",particle.pz)  
                print("spin:",particle.spin)
            
            ##-- Gluons (GF only)
            if(pdgId == 21 and ("GF" in gridpack_Type) and not foundFirstGluon):
                foundFirstGluon = 1 
                g1 = particle 
            elif(pdgId == 21 and ("GF" in gridpack_Type) and foundFirstGluon):
                g2 = particle  
         
            ##-- Spin-0 or Spin-2 Resonant Particle, X->YH resonant particle
            if(pdgId == 45):
                X = particle 
                
            ##-- Higgs Bosons 
            if(pdgId == 25):
                H1 = particle 
            if(pdgId == 35):
                H2 = particle 
                
        ##-- Determine Leading, Subleading particles based on pT, save variables 
        for p in doubleParticles:
            p1 = eval("%s1"%(p))
            p2 = eval("%s2"%(p))
            p1_pt = GetPt(p1)
            p2_pt = GetPt(p2)
            if(p1_pt > p2_pt):
                exec("Lead_%s = p1"%(p))
                exec("Sublead_%s = p2"%(p))
            elif(p2_pt > p1_pt):
                exec("Lead_%s = p2"%(p))
                exec("Sublead_%s = p1"%(p)) 
            else:
                exec("Lead_%s = p1"%(p))
                exec("Sublead_%s = p2"%(p))   
            
            ##-- Save variables 
            for v in variables:
                
                ##-- pt gets special treatment
                if(v == "pt"):
                    ##-- Leading variable expression, subleading variable expression
                    exec("d['Lead_%s_%s'].append(p1_pt)"%(p, v))
                    exec("d['Sublead_%s_%s'].append(p2_pt)"%(p, v))
                else:
                    ##-- Leading variable expression, subleading variable expression
                    lvExp = "Lead_%s.%s"%(p, v)
                    slvExp = "Sublead_%s.%s"%(p, v)
                    exec("Lead_%s_%s = %s"%(p, v, lvExp))
                    exec("Sublead_%s_%s = %s"%(p, v, slvExp))
                    exec("d['Lead_%s_%s'].append(Lead_%s_%s)"%(p, v, p, v))
                    exec("d['Sublead_%s_%s'].append(Sublead_%s_%s)"%(p, v, p, v))                    
                
        ##-- Resonant particle
        X_pt = GetPt(X)
        for v in variables:
            c = "X_%s"%(v)
            if(v == "pt"):
                X_pt = GetPt(X)
            else:
                exec("%s = X.%s"%(c, v))
            exec("d['%s'].append(%s)"%(c, c))
            
        ##-- Resonant particle eta and phi 
        X_v = TLorentzVector() 
        X_v.SetPxPyPzE(X_px, X_py, X_pz, X_e)
        X_eta = X_v.Eta()
        X_phi = X_v.Phi()   
        d['X_eta'].append(X_eta)
        d['X_phi'].append(X_phi)
        
        ##-- DeltaR between pairs of particles 
        for p_Name in doubleParticles:
            Lead_p_v = TLorentzVector() ##-- Lead particle vector
            Sublead_p_v = TLorentzVector() ##-- Subleading particle vector 
            
            ##-- Set pxpypzE of TLorentzVector in order to get Eta and Phi 
            for rank in ["Lead", "Sublead"]:
                for kinVar in ["px", "py", "pz", "e"]:
                    exec("%s_p_%s = %s_%s_%s"%(rank, kinVar, rank, p_Name, kinVar))

            Lead_p_v.SetPxPyPzE(Lead_p_px, Lead_p_py, Lead_p_pz, Lead_p_e)
            Sublead_p_v.SetPxPyPzE(Sublead_p_px, Sublead_p_py, Sublead_p_pz, Sublead_p_e)

            ##-- Get Eta and Phi 
            Lead_p_eta = Lead_p_v.Eta()
            Lead_p_phi = Lead_p_v.Phi()
            Sublead_p_eta = Sublead_p_v.Eta()
            Sublead_p_phi = Sublead_p_v.Phi()
            
            Deta = float(Lead_p_eta - Sublead_p_eta)
            Dphi = float(Lead_p_phi - Sublead_p_phi)

            for rank in ["Lead", "Sublead"]:
                for ang_var in ["eta", "phi"]:
                    exec("d['%s_%s_%s'].append(%s_p_%s)"%(rank, p_Name, ang_var, rank, ang_var))

            DR = GetDR(Lead_p_eta, Sublead_p_eta, Lead_p_phi, Sublead_p_phi)

            if(p_Name == "H"):
                d['DR_HH'].append(DR)
                d['HH_Deta'].append(Deta)
                d['HH_Dphi'].append(Dphi)
            elif(p_Name == "Outgoing_q"):
                d['DR_OutqOutq'].append(DR)
                d['OutqOutq_Deta'].append(Deta)
                d['OutqOutq_Dphi'].append(Dphi)  
            elif(p_Name == "Incoming_q"):
                d['DR_InqInq'].append(DR)
                d['InqInq_Deta'].append(Deta)
                d['InqInq_Dphi'].append(Dphi)                  
        
        ##-- Append event pdgIds
        d['pdgIds'].append(pdgIds)


On file: LHE_Files/GF_XYH/cmsgrid_final.lhe
Looping Events ...
On Event 0
STOPPING at 100 events


In [11]:
    ##-- Useful debug is making sure length of each variable list is the same after looping events 
    if(debug):
        for p in particle_Names:
            for v in variables:
                columnName = "%s_%s"%(p,v)
                print("columnName:",columnName)
                print("len(%s):"%(columnName),len(d[columnName]))          
        
    ##-- After event loop, save dataframe
    df = pd.DataFrame(data = d)
    if(debug): print(df)
    pickle.dump(df, open( "%s.p" % (gridpack_Type), "wb" )) ##-- Pickle the dataframe 
                
print("DONE")

DONE


In [12]:
##-- To check a dataframe 
df = pickle.load( open( "GF_XYH.p", "rb" ) )
df

,X_px,X_py,X_pz,X_e,X_m,X_spin,X_status,X_lifetime,X_pt,Lead_H_px,...,Sublead_H_eta,Sublead_H_phi,Lead_g_eta,Lead_g_phi,Sublead_g_eta,Sublead_g_phi,pdgIds,DR_HH,HH_Deta,HH_Dphi
0,0.0,0.0,1267.659801,1302.674693,299.999975,0.0,2.0,0.0,0.0,-8.438133,...,4.894167,-0.826219,1.000000e+11,0.0,-1.000000e+11,0.0,"[21.0, 21.0, 45.0, 35.0, 25.0]",3.206904,-0.643915,3.141593
1,0.0,0.0,-637.756927,704.792936,299.998640,0.0,2.0,0.0,0.0,-8.424586,...,-3.556218,1.192892,1.000000e+11,0.0,-1.000000e+11,0.0,"[21.0, 21.0, 45.0, 35.0, 25.0]",3.183686,0.515999,-3.141593
2,0.0,0.0,199.789880,360.439137,300.000626,0.0,2.0,0.0,0.0,18.798092,...,2.158722,-2.342328,1.000000e+11,0.0,-1.000000e+11,0.0,"[21.0, 21.0, 45.0, 35.0, 25.0]",3.155648,-0.297504,3.141593
3,0.0,0.0,-35.174196,302.052177,299.997156,0.0,2.0,0.0,0.0,-15.316420,...,-1.554340,-0.544363,1.000000e+11,0.0,-1.000000e+11,0.0,"[21.0, 21.0, 45.0, 35.0, 25.0]",3.643778,1.845945,3.141593
4,0.0,0.0,-128.957956,326.547088,300.004744,0.0,2.0,0.0,0.0,-19.167158,...,-1.735266,0.780548,1.000000e+11,0.0,-1.000000e+11,0.0,"[21.0, 21.0, 45.0, 35.0, 25.0]",3.153761,0.276775,-3.141593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,711.721948,772.365109,299.999551,0.0,2.0,0.0,0.0,-13.343008,...,3.440439,-0.988398,1.000000e+11,0.0,-1.000000e+11,0.0,"[21.0, 21.0, 45.0, 35.0, 25.0]",3.144063,-0.124610,3.141593
96,0.0,0.0,-1006.678295,1050.429066,300.000057,0.0,2.0,0.0,0.0,-13.289854,...,-3.761762,1.019732,1.000000e+11,0.0,-1.000000e+11,0.0,"[21.0, 21.0, 45.0, 35.0, 25.0]",3.146114,0.168600,-3.141593
97,0.0,0.0,-122.850962,324.179548,300.000035,0.0,2.0,0.0,0.0,1.353485,...,-2.122854,1.634035,1.000000e+11,0.0,-1.000000e+11,0.0,"[21.0, 21.0, 45.0, 35.0, 25.0]",3.258260,0.864090,-3.141593
98,0.0,0.0,536.655816,614.823577,300.014277,0.0,2.0,0.0,0.0,13.436127,...,3.366922,-2.384761,1.000000e+11,0.0,-1.000000e+11,0.0,"[21.0, 21.0, 45.0, 35.0, 25.0]",3.141598,0.005719,3.141593
